<a href="https://colab.research.google.com/github/Niklas123456789/DM/blob/main/task2/generate_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#preinstalled version of pytorch has to be the same as the pre-compiled versions of the pytorch-geometric packages that we download later on.
#versions might change quickly, so if you get a strange error later on, check the torch version of Google colab later on as follows:

import torch
torch.__version__

'1.7.0+cu101'

In [2]:
# Script to generate variations of the kernels yourself
# https://ucloud.univie.ac.at/index.php/s/E3YKph0jkpbw8TN


# #Download the TUDataset Repository with
!git clone https://github.com/chrsmrrs/tudataset.git
# #move this script to tudataset/tud_benchmark

# #Install pytorch geometric: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
# #Here is the gpu cuda installation, for the cpu version replace cu102 with cpu
%pip --no-cache-dir install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
%pip --no-cache-dir install torch-geometric

%pip --no-cache-dir install networkit

%pip --no-cache-dir install pybind11
!sudo apt-get install libeigen3-dev

%cd ..
%cd /content/tudataset/tud_benchmark/kernel_baselines/
! ls
! g++ -I /usr/include/eigen3 -O3 -shared -std=c++11 -fPIC `python3 -m pybind11 --includes`  kernel_baselines.cpp src/*cpp -o ../kernel_baselines`python3-config --extension-suffix`
%cd ..

!ls -al /usr/local/cuda

Cloning into 'tudataset'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (485/485), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 3344 (delta 244), reused 300 (delta 114), pack-reused 2859
Receiving objects: 100% (3344/3344), 8.47 MiB | 17.96 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 2.7MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 1.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 1.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 3.0MB/s 
     |████████████████████████████████| 194kB 4.3MB/s 
     |████████████████████████████████| 235kB 16.1MB/s 
     |████████████████████████████████| 2

In [26]:
import os
import torch
import numpy as np
import kernel_baselines as kb
import auxiliarymethods
import auxiliarymethods.auxiliary_methods as aux
import auxiliarymethods.kernel_evaluation as ke
import auxiliarymethods.datasets as dp
from scipy.sparse import save_npz
from itertools import combinations

import networkit as nk
import networkx as nx
from auxiliarymethods.reader import tud_to_networkx

In [4]:
def setup_directory(dir_name, verbose=False):
    """
    Setup directory in case it does not exist
    Parameters:
    -------------
    dir_name: str, path + name to directory
    verbose: bool, indicates whether directory creation should be printed or not.
    """
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name)
            if verbose:
                print("Created Directory: {}".format(dir_name))
        except Exception as e:
            raise RuntimeError(
                "Could not create directory: {}\n {}".format(dir_name, e))


In [5]:
use_edge_labels = False
for USE_LABELS in [True, False]:# Except IMDB-BINARY
    break
    for dataset, use_labels in [["IMDB-BINARY", False],["MSRC_21",USE_LABELS], ["NCI1", USE_LABELS], ["ENZYMES", USE_LABELS]]:
        if use_labels:
            base_path = os.path.join("kernels","node_labels")
        else:
            base_path = os.path.join("kernels","without_labels")
        setup_directory(base_path)
        print("Start processing data set ", dataset)
        # Download dataset.
        classes = dp.get_dataset(dataset)
        # *Weisfeihler-Lehman*
        print("Start computing Weisfeihler-Lehman gram matrix and vector representations")
        iterations = 6
        #0 taking just the nodelabels themselves into account; 
        #1 considers nearest-neighbours, 2 one layer deeper and so on
        for i in range(1, iterations):
            print("Start iteration ", i)
            #Gram Matrix for the Weisfeiler-Lehman subtree kernel
            gram_matrix_wl = kb.compute_wl_1_dense(dataset, i, use_labels, use_edge_labels)
            np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_wl{i}.csv"),
                    gram_matrix_wl,
                    delimiter=";")
            #Sparse Vectors for the Weisfeiler-Lehmann subtree kernel
            vectors_wl = kb.compute_wl_1_sparse(dataset, i, use_labels, use_edge_labels)
            save_npz(os.path.join(base_path,f"{dataset}_vectors_wl{i}.npz"),
                    vectors_wl, compressed=True)

        # *Graphlet kernel*
        print("Start computing Graphlet gram matrix")

        #Gram Matrix for the Graphlet kernel
        gram_matrix_graphlet= kb.compute_graphlet_dense(dataset, use_labels, use_edge_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_graphlet.csv"),
                gram_matrix_graphlet,
                delimiter=";")

        print("Start computing Graphlet vector representation")
        #Sparse Vectors for the Graphlet kernel
        vectors_graphlet = kb.compute_graphlet_sparse(dataset, use_labels, use_edge_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_graphlet.npz"),
                vectors_graphlet, compressed=True)


        print("Start computing Shortest path gram matrix")

        #Gram Matrix for the Shortest path kernel
        gram_matrix_shortestpath = kb.compute_shortestpath_dense(dataset, use_labels)
        np.savetxt(os.path.join(base_path,f"{dataset}_gram_matrix_shortestpath.csv"),
                gram_matrix_shortestpath,
                delimiter=";")

        print("Start computing Shortest path vector representation")

        #Sparse Vectors for the Shortest path kernel
        vectors_shortestpath = kb.compute_shortestpath_sparse(dataset, use_labels)
        save_npz(os.path.join(base_path,f"{dataset}_vectors_shortestpath.npz"),
                vectors_shortestpath, compressed=True)



In [71]:
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def density_list(G):
  return [2*g.number_of_edges()/(g.number_of_nodes() * (g.number_of_nodes()-1)) for g in G]

def oep(graph):
    try:
        return next(nx.optimize_edit_paths(G[0],graph,timeout=0.05))[2]
    except StopIteration:
        return np.inf

# simple community modularity kernel
def community_modularity_kernel(G):
  comms = [greedy_modularity_communities(g) for g in G]
  mods = [modularity(g,greedy_modularity_communities(g)) for g in G]

  mat = np.zeros((1000,1000))

  for i in range(1000):
    for j in range(1000):
      mat[i,j] = np.minimum(mods[i],mods[j]) / np.maximum(mods[i],mods[j])

  return np.nan_to_num(mat)

# simple kernel from graph densities
def density_kernel(G):
  dens = density_list(G)
  return np.array([dens]).T.dot(np.array([dens]))

def eigencentrality_kernel(G):
  eigencents = []
  mat = np.zeros((1000,1000))
  
  for g in G:
    vals = nx.eigenvector_centrality(g)
    eigencents.append(np.array([sorted(vals.values(),reverse=True)[:5]]))
  
  for i in range(1000):
    for j in range(i,1000):
      mat[i,j] = eigencents[i].dot(eigencents[j].T)
  
  return np.maximum(mat, mat.T)

def isomorphism_kernel(G):
    '''
    This function returns the number of isomorphic pairs of a network with several ego-graphs.
    '''
    iso = np.diag(np.full(len(G),1)) # graphs are always isomorphic to themselves

    for i in range(1, 1000):
      for j in range(i+1,1000):
        iso[i,j] = np.where(nx.faster_could_be_isomorphic(G[i], G[j]),1,0)
    return np.maximum(iso, iso.T)

In [38]:
dataset = "IMDB-BINARY"
classes = dp.get_dataset(dataset)
G = tud_to_networkx(dataset)
nkG = [nk.nxadapter.nx2nk(g) for g in G]

def eval(matrices):
  accuracy, std_10, std_100 = ke.kernel_svm_evaluation(matrices, classes, num_repetitions=num_reps, all_std=True)
  print(accuracy, std_10, std_100)

In [72]:
all_kernels = []
num_reps = 10
all_kernels.append(density_kernel(G))
all_kernels.append(community_modularity_kernel(G))
# print(accuracy, std_10, std_100) == (57.489999999999995, 1.2061923561356205, 5.356295361534873) # density, community

eval(all_kernels)
all_kernels.append(isomorphism_kernel(G))
eval(all_kernels)
# print(accuracy, std_10, std_100) == (59.03000000000001 1.8947559209565754 5.689384852512616) # density, community, isomorphic
all_kernels.append(density_kernel(G))
eval(all_kernels)
# print(accuracy, std_10, std_100) == (62.17999999999999 1.0505236789335117 4.883400454601282) # density, community, isomorphic, eigencentrality
all_kernels.append(eigencentrality_kernel(G))
eval(all_kernels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


58.06 0.7952358140828414 4.46277940301781
59.75 1.3462912017836253 5.6415866562519446
60.160000000000004 1.4602739469017447 4.694081379780286
62.52 1.2327205684987992 4.788486190854057


Experiments below

In [15]:
all_matrices = []
num_reps = 10
use_labels, use_edge_labels = False, False

# 1-WL kernel, number of iterations in [1:6].
for i in range(1,6):
    gm = kb.compute_wl_1_dense(dataset, i, use_labels, use_edge_labels)
    # Cosine normalization.
    gm = aux.normalize_gram_matrix(gm)
    all_matrices.append(gm)
    print(gm.shape)
    print(gm[:5,:5])

# accuracy is a mean of means of test accuracies
accuracy, std_10, std_100 = ke.kernel_svm_evaluation(all_matrices, classes, num_repetitions=num_reps, all_std=True)
print(accuracy, std_10, std_100)

(1000, 1000)
[[1.         0.09855258 0.08595703 0.22520897 0.13533299]
 [0.09855258 1.         0.07596444 0.2003595  0.04272494]
 [0.08595703 0.07596444 1.         0.21237361 0.05681146]
 [0.22520897 0.2003595  0.21237361 1.         0.10203572]
 [0.13533299 0.04272494 0.05681146 0.10203572 1.        ]]
71.96000000000001 0.798999374217527 4.28933561288925


In [ ]:
G1 = nx.cycle_graph(6)
G2 = nx.wheel_graph(7)
nx.graph_edit_distance(G2, G1)

import time

start = time.time()
for g1 in range(5):
  for g2 in range(5):
    gen = nx.optimize_edit_paths(G[g1],G[g2],timeout=0.05)

    try:
      print(next(gen)[2])
    except StopIteration:
      pass
      
print(time.time() - start)

98.0
185.0
97.0
185.0
91.0
185.0
180.0
190.0
105.0
184.0
102.0
190.0
94.0
173.0
190.0
184.0
93.0
190.0
92.0
186.0
44.0
0.9439151287078857


In [ ]:
from tqdm import tqdm

def is_symmetric(a, rtol=1e-05, atol=1e-08):
    return numpy.allclose(a, a.T, rtol=rtol, atol=atol)

dists = np.zeros((1000,1000))
print(dists.shape)


start = time.time()
dists[0] = [oep(g) for g in G]
print(time.time()-start)

# for g1 in range(1,len(G)):
#   print(g1)
#   for g2 in range(g1,len(G)):
#     if g1 != g2:
#       gen = nx.optimize_edit_paths(G[g1],G[g2],timeout=0.05)

#       try:
#         dists[g1,g2] = next(gen)[2]
#       except StopIteration:
#         dists[g1,g2] = np.inf

# print(is_symmetric(dists))
# ke.kernel_svm_evaluation([dists], classes, num_repetitions=num_reps, all_std=True)

(1000, 1000)
24.982826709747314
